In [ ]:
from fastapi import FastAPI, Body
import uvicorn
import nest_asyncio
import numpy as np
import pandas as pd
import re
import tensorflow as tf
import nltk
from transformers import BertTokenizer, BertForSequenceClassification
from torch.utils.data import Dataset, DataLoader
import torch

nest_asyncio.apply()
app = FastAPI()

print("PyTorch 버전:", torch.__version__)
print("CUDA 사용 가능 여부:", torch.cuda.is_available())
print("PyTorch에서 사용하는 CUDA 버전:", torch.version.cuda)
torch.set_default_device("cuda")
# GPU용 Generator 생성
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
generator = torch.Generator(device=device)

# BERT 토크나이저
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")

model = BertForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=3)
state_dict = torch.load("final_model_weights_filt.pth", weights_only=True)  # 가중치 로드
model.load_state_dict(state_dict)

# 모델 평가 모드 전환
model.eval()

@app.post("/classification")
async def classification(body: dict = Body(...)):
    # 입력 문장
    input_text = body.get("text")
    print(input_text)
    
    # 토큰화
    input_encoding = tokenizer(
        input_text,
        truncation=True,
        padding=True,
        max_length=128,
        return_tensors="pt"
    )
    
    # 모델 예측
    with torch.no_grad():
        # 입력 데이터 추출
        input_ids = input_encoding["input_ids"]
        attention_mask = input_encoding["attention_mask"]
    
        # 모델 예측 수행
        outputs = model(input_ids=input_ids, attention_mask=attention_mask)
        logits = outputs.logits  # [1, num_classes]
        
        # 소프트맥스 확률 계산
        probabilities = torch.softmax(logits, dim=-1)
        
        # 가장 큰 확률의 클래스 인덱스 예측
        prediction = torch.argmax(probabilities, dim=-1).item()
        print(prediction)
        return {"result": prediction}

uvicorn.run(app, host="0.0.0.0", port=8000)

PyTorch 버전: 2.5.1+cu118
CUDA 사용 가능 여부: True
PyTorch에서 사용하는 CUDA 버전: 11.8


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
INFO:     Started server process [27108]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://0.0.0.0:8000 (Press CTRL+C to quit)


fuck you
2
INFO:     127.0.0.1:62688 - "POST /classification HTTP/1.1" 200 OK
